---
## The Perceptron

The *perceptron* is a neuron model. It performs binary classification that map an input feature to an output decision (usually classifying data into one of two categories e.g -1 or 1)

For the purposes of this notebook, we will consider only one type of perceptron:

### **Single-Layer** Perceptron:
   - Simple to implement.
   - Limited to linearly separable patterns.   
---

### Components of Perceptrons
1. Weights
2. Activation Function
3. Learning Algorithm 

Perceptrons are given inputs $x_i$.

Each *input* $x_i$ and the *bias* $b$ is mapped to a particular *weight* $w_i$. 

A *weighted sum* is calculated, with the formula: $$z = x_i*w_i + b \quad\forall i $$

This z is input to the **Activation Function**, which can produce two possible outputs. These outputs divide the input data into two distinct categories.

The **Learning Algorithm** updates the weights of the perceptron depending on how large the difference between the actual and predicted outputs (error) is. 

The biases and weights are updated:

\begin{align}
\text{bias} &\leftarrow \text{bias} * \eta \; \text{predicted} - \text{actual} \\
w_i &\leftarrow w_i * \eta \; (\text{predicted} - \text{actual}) * x_i \quad \forall i
\end{align}


In [108]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from mlxtend.plotting import plot_decision_regions

sns.set_theme
df = pd.read_csv("https://raw.githubusercontent.com/RandyRDavila/Data_Science_and_Machine_Learning_Spring_2022/main/Lecture_3/Datasets/iris_dataset.csv")

In [4]:
df.iloc[:100]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
95,5.7,3.0,4.2,1.2,versicolor
96,5.7,2.9,4.2,1.3,versicolor
97,6.2,2.9,4.3,1.3,versicolor
98,5.1,2.5,3.0,1.1,versicolor


In [106]:
class SL_Perceptron(object):
    """
    A class used to represent a Single-Layer Perceptron

    Attributes
    ----------
    inputs : array
        an array of inputs to the perceptron
    outputs : array
        an array of expected outputs from the perceptron
    eta : float
        the 'penalty' for an incorrect output
    epochs : int
        the number of epochs to train

    Methods
    -------
    train()
        trains the perceptron for self.epoch iterations, to produce self.outputs from self.inputs.

    train_err_limit(self, percent_err, max_epochs=None)
        trains the perceptron until it reaches a certain percent_error rate or until max_epochs are reached.

    update_weights()
        updates the weights of the perceptron(self.w_) once using self.inputs and comparing with self.outputs.

    net_input(inputs):
        calculates the net input to the perceptron, by multiplying a given input with the weights and adding the bias.
    
    predict():
        calculates the output the perceptron produces for a given input.
    
    """
    def __init__(self, inputs, outputs, eta = 0.5, epochs = 100):
        self.eta = eta
        self.epochs = epochs
        self.inputs = inputs
        self.outputs = outputs
        self.w_ = np.random.rand(1+inputs.shape[1])

    def train(self):
        epochs = self.epochs
        self.errors_ = []

        for _ in range(epochs):            
            errors = self.update_weights()
            if errors == 0:
                return self
            else:
                self.errors_.append(errors)
                
        return self

    def train_err_limit(self, percent_err, max_epochs=None):
        if max_epochs == None:
            max_epochs = self.epochs
        if max_epochs < 1:
            raise ValueError("max_epoch must be positive")
        if not(0 < percent_err < 1):
            raise ValueError("percent_err must be between 0 and 1")
            
        
        epochs = 0
        self.errors_ = []
        curr_percent_err = 1;
        
        while (max_epochs > epochs and curr_percent_err > percent_err):
            errors = self.update_weights()
            curr_percent_err = errors / self.outputs.shape[0]
            self.errors_.append(errors)
            epochs+=1
        
        return epochs, curr_percent_err
            
    def update_weights(self):
        errors = 0
        
        for xi, target in zip(self.inputs, self.outputs):
            prediction = self.predict(xi)
            error = prediction - target
            
            if (error != 0):
                update = self.eta * error
                self.w_[:-1] -= update*xi
                self.w_[-1] -= update
                errors += 1
                
        return errors
    
    def net_input(self, inputs):
        return (inputs @ self.w_[:-1]) + self.w_[-1]

    def predict(self, inputs):
        return np.where(self.net_input(inputs) >= 0, 1, -1)               

In [109]:
X = df[["sepal_length", "petal_length"]].iloc[:100].values
y = df.iloc[:100].species.values
y = np.where(y == "setosa", -1, 1)

clf = SL_Perceptron(X, y, epochs=1_000_000)
y_hat = clf.predict(X)
print(y == y_hat)

num_errors = clf.train_err_limit(0.002, 60)
print("The num epochs and error percentage output is: " + str(num_errors))
y_hat = clf.predict(X)
print(y == y_hat)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]
The num epochs and error percentage output is: (6, 0.0)
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True

In [111]:
import os
os.getcwd()

'C:\\Users\\ism_s\\Desktop\\RICE_Stuff\\Cmor438\\PythonPrac'